In [12]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import concurrent.futures
import multiprocessing
MAX_THREADS = 30

In [13]:
import pyarrow.feather as feather
df = feather.read_feather('data/artsy_full.feather')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27577 entries, 0 to 27576
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    27577 non-null  object 
 1   slug                  27577 non-null  object 
 2   title                 27576 non-null  object 
 3   category              27576 non-null  object 
 4   medium                27575 non-null  object 
 5   date                  27577 non-null  object 
 6   iconicity             27577 non-null  float64
 7   sold                  27577 non-null  bool   
 8   image_versions        27469 non-null  object 
 9   dimensions_height_in  14120 non-null  float64
 10  dimensions_width_in   13607 non-null  float64
 11  dimensions_depth_in   836 non-null    float64
 12  permalink             27577 non-null  object 
 13  api_link              27577 non-null  object 
 14  thumb_link            26482 non-null  object 
 15  image_link         

In [15]:
df.isna().sum()

id                          0
slug                        0
title                       1
category                    1
medium                      2
date                        0
iconicity                   0
sold                        0
image_versions            108
dimensions_height_in    13457
dimensions_width_in     13970
dimensions_depth_in     26741
permalink                   0
api_link                    0
thumb_link               1095
image_link                108
genes                       0
artist                   2891
artist_slug              2891
artist_id                2891
dtype: int64

In [16]:
import requests

def download_img(row):
    filename = f'{row.id}.jpg'
    try:
        img_data = requests.get(row.image_link.replace('{image_version}', 'large'))
        with open('images/large/'+ filename, 'wb') as handler:
            handler.write(img_data.content)
        row['filename'] = filename
        
    except:
        row['filename'] = float('nan')
        
    return row

count = 0
def download_img_with_index(index):
    row = df.iloc[index]
    filename = f'{row.id}.jpg'
    try:
        img_data = requests.get(row.image_link.replace('{image_version}', 'large'))
        with open('images/large/'+ filename, 'wb') as handler:
            handler.write(img_data.content)
        row['filename'] = filename
        
    except:
        row['filename'] = float('nan')
        
    print(index, end='\r')
    count += 1
        
    df.iloc[index] = row



In [17]:
pd.options.mode.chained_assignment = None

indices = df.index

threads = min(MAX_THREADS, len(indices))
count = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_img_with_index, indices)

In [18]:
import pyarrow.feather as feather

In [20]:
feather.write_feather(df, 'artsy_full_images.feather')